In [1]:
import os
from pathlib import Path

from IPython.display import Code

from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform
from pydrake.multibody.meshcat import ContactVisualizer, ContactVisualizerParams
from pydrake.multibody.parsing import PackageMap, Parser
from pydrake.multibody.plant import AddMultibodyPlant, MultibodyPlantConfig
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import ApplyVisualizationConfig, ModelVisualizer, VisualizationConfig

In [2]:
meshcat = StartMeshcat()


INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
bell_pepper_url = "package://drake_models/veggies/yellow_bell_pepper_no_stem_low.sdf"
bell_pepper_str = Path(PackageMap().ResolveUrl(bell_pepper_url)).read_text(encoding="utf-8")
Code(bell_pepper_str, language="xml")

<?xml version="1.0"?>
<sdf version="1.7" xmlns:drake="drake.mit.edu">
  <model name="yellow_bell_pepper_no_stem">
    <link name="yellow_bell_pepper_no_stem">
      <inertial>
        <pose>0.000537 -0.00272 0.0384 0 0 0</pose>
        <mass>0.159</mass>
        <inertia>
          <ixx> 0.000101</ixx>
          <ixy>-0.000001</ixy>
          <ixz>-0.000004</ixz>
          <iyy> 0.000105</iyy>
          <iyz> 0.000007</iyz>
          <izz> 0.000107</izz>
        </inertia>
      </inertial>
      <visual name="visual">
        <geometry>
          <mesh>
            <uri>package://drake_models/veggies/assets/yellow_bell_pepper_no_stem_low.obj</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
      </visual>
      <collision name="collision">
        <geometry>
          <mesh>
            <uri>package://drake_models/veggies/assets/yellow_bell_pepper_no_stem_low.vtk</uri>
            <scale>1 1 1</scale>
          </mesh>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>5.0e4</drake:hydroelastic_modulus>
          <!-- Most shapes (capsule, cylinder, ellipsoid, sphere) need
            drake:mesh_resolution_hint, but the resolution hint is no-op
            for the mesh geometry. That's why we do not set it here. -->
          <drake:hunt_crossley_dissipation>10</drake:hunt_crossley_dissipation>
          <!-- Both mu_dynamic and mu_static are used in Continuous system.
            Only mu_dynamic is used in Discrete system.  -->
          <drake:mu_dynamic>0.5</drake:mu_dynamic>
          <drake:mu_static>1.0</drake:mu_static>
        </drake:proximity_properties>
      </collision>
    </link>
    <frame name="origin">
      <pose relative_to="yellow_bell_pepper_no_stem">0 0 0 0 0 0</pose>
    </frame>
    <frame name="flush_bottom_center__z_up">
      <pose relative_to="yellow_bell_pepper_no_stem">0 0 0 0 0 0</pose>
    </frame>
  </model>
</sdf>

In [24]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.AddModels(url=bell_pepper_url)
visualizer.Run(loop_once=False)

INFO:drake:Click 'Stop Running' or press Esc to quit


In [25]:
bowl_url = "package://drake_models/dishes/evo_bowl_compliant.urdf"
bowl_str = Path(PackageMap().ResolveUrl(bowl_url)).read_text(encoding="utf-8")
Code(bowl_str, language="xml")

<?xml version="1.0"?>
<robot xmlns:drake="http://drake.mit.edu" name="bowl">
  <link name="bowl">
    <inertial>
      <mass value="0.5"/>
      <origin xyz="-6.8577139059790044e-06 -1.7473168538940867e-05 -0.003065665933405878"/>
      <inertia ixx="0.0009715521141455593"
               ixy="-1.8443983471817828e-07"
               ixz="2.4082416205698527e-08"
               iyy="0.0009793965751030533"
               iyz="6.72877272895659e-08"
               izz="0.001592061359500251"/>
    </inertial>
    <visual name="visual">
      <geometry>
        <mesh filename="package://drake_models/dishes/assets/evo_bowl_no_mtl.obj"/>
      </geometry>
      <material>
       <color rgba="0.9 0.8 0.7 0.5"/>
      </material>
    </visual>
    <collision name="collision">
      <geometry>
        <mesh filename="package://drake_models/dishes/assets/evo_bowl_fine44k.vtk"/>
      </geometry>
      <drake:proximity_properties>
        <drake:compliant_hydroelastic/>
        <drake:hydroelastic_modulus value="1e7"/>
        <drake:mu_dynamic value="0.5"/>
        <drake:hunt_crossley_dissipation value="1.25"/>
      </drake:proximity_properties>
    </collision>
  </link>
</robot>

In [6]:
test_mode = "TEST_SRCDIR" in os.environ
if test_mode:
    bowl_str = bowl_str.replace("evo_bowl_fine44k.vtk", 
                                "evo_bowl_coarse3k.vtk")

In [7]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModels(file_contents=bowl_str, file_type="urdf")
visualizer.Run(loop_once=True)

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html



In [9]:
table_top_url = "package://drake_models/dishes/table_top.urdf"
table_top_str = Path(PackageMap().ResolveUrl(table_top_url)).read_text(encoding="utf-8")
Code(table_top_str, language="xml")

<?xml version="1.0"?>
<robot name="table_top" xmlns:drake="http://drake.mit.edu">
  <link name="table_top_link">
    <visual name="visual">
      <geometry>
        <box size="0.6 1.0 0.05"/>
      </geometry>
      <material>
       <color rgba="0.9 0.8 0.7 0.5"/>
      </material>
    </visual>
    <collision name="collision">
      <geometry>
        <box size="0.6 1.0 0.05"/>
      </geometry>
      <drake:proximity_properties>
        <drake:compliant_hydroelastic/>
        <drake:hydroelastic_modulus value="1e7"/>
        <drake:mu_dynamic value="0.5"/>
        <drake:hunt_crossley_dissipation value="1.25"/>
      </drake:proximity_properties>
    </collision>
  </link>
  <frame name="top_surface" link="table_top_link" xyz="0 0 0.025" rpy="0 0 0"/>
</robot>

In [10]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.AddModels(url=table_top_url)
visualizer.Run(loop_once=True)

In [20]:
def add_scene(time_step):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlant(
        MultibodyPlantConfig(
            time_step=time_step,
            discrete_contact_approximation="lagged"),
        builder)
    parser = Parser(plant)

    # Load the assets that we created.
    parser.AddModels(url=bell_pepper_url)
    parser.AddModels(file_contents=bowl_str, file_type="urdf")
    # parser.AddModels(url=table_top_url)

    # Weld the table top to the world so that it's fixed during simulation.
    # The top surface passes the world's origin.
    # plant.WeldFrames(plant.world_frame(), 
                    #  plant.GetFrameByName("top_surface"))

    # Finalize the plant after loading the scene.
    plant.Finalize()

    # Place the bowl on top of the table.
    X_WB = RigidTransform(p=[0, 0, 0])
    plant.SetDefaultFreeBodyPose(plant.GetBodyByName("bowl"), X_WB)
    
    # Drop the bell pepper from above the rim of the bowl. 
    X_WC = RigidTransform(p=[-0.06, 0, 0.30])
    plant.SetDefaultFreeBodyPose(plant.GetBodyByName("yellow_bell_pepper_no_stem"), X_WC)

    return builder, plant

In [21]:
def add_viz(builder, plant):
    ApplyVisualizationConfig(
        builder=builder, meshcat=meshcat,
        config=VisualizationConfig(
                 publish_contacts=False))    
    ContactVisualizer.AddToBuilder(
        builder=builder, plant=plant, meshcat=meshcat,
        params=ContactVisualizerParams(
                 newtons_per_meter=20,
                 newton_meters_per_meter= 0.1))

In [22]:
meshcat.Delete()
meshcat.DeleteAddedControls()

time_step = 1e-2
builder, plant = add_scene(time_step)
add_viz(builder, plant)

diagram = builder.Build()

simulator = Simulator(diagram)

# In interactive mode, simulate for longer time.
# In test mode, simulate for shorter time.
sim_time = 2 if not test_mode else 0.01

# meshcat.StartRecording()
simulator.set_target_realtime_rate(1)
simulator.AdvanceTo(2)
# meshcat.StopRecording()
# meshcat.PublishRecording()

==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html

